In [1]:
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, precision_recall_curve, roc_curve, roc_auc_score, f1_score
from sklearn.metrics import average_precision_score
import pandas as pd
import numpy as np
from copy import deepcopy
import seaborn as sns
from sklearn.metrics import classification_report,average_precision_score, precision_recall_curve, roc_curve, roc_auc_score, f1_score, confusion_matrix, auc
import time
from sklearn.model_selection import GroupKFold, StratifiedKFold, GridSearchCV, train_test_split, RandomizedSearchCV
import matplotlib.pyplot as plt
import sklearn
from copy import deepcopy
from sklearn.model_selection import cross_val_score
import xgboost as xgb
# from xgboost import XGBClassifier, plot_importance
import warnings
warnings.filterwarnings('ignore')

2023-02-01 18:55:50.983127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 18:55:51.117895: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-01 18:55:51.135999: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/user/.mujoco/mujoco200/bin:/home/user/.mujoco/mujoco200/bin
2023-02-01 18:55:51.136010:

### Loading Data + Visualization

In [2]:
# df = pd.read_csv('./data_collection4/ep1.csv')
# df = pd.read_csv('./data_collection4/paired_errors.csv')
# df = pd.read_csv('./robot_data/merged_robot_data.csv')
# df = pd.read_csv('./data_collection4/merged_sim_data.csv')
df = pd.read_csv('./robotdatacollection3/merged_robot_data_100episodes.csv')


In [3]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 't', 'x', 'y', 'z', 'rx', 'ry', 'rz',
       'vx', 'vy', 'vz', 'Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz', 'Case'],
      dtype='object')

In [4]:
df.shape

(488389, 19)

In [5]:
# df = df.drop(['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0','t'], axis=1)
df = df.drop(['Unnamed: 0.1','Unnamed: 0','t'], axis=1)

# df = df.drop(['Unnamed: 0','t'], axis=1)

In [6]:
# df = pd.read_feather('./data_collection4/paired_errors.feather')

In [7]:
df.columns

Index(['x', 'y', 'z', 'rx', 'ry', 'rz', 'vx', 'vy', 'vz', 'Fx', 'Fy', 'Fz',
       'Mx', 'My', 'Mz', 'Case'],
      dtype='object')

In [8]:
tot = len(df.Case)
pos = df.Case.sum()
neg = tot-pos
print('Total data: ', tot)
print(f'Label: 0 {neg} | {(neg/tot)*100}%')
print(f'Label: 1 {pos} | {(pos/tot)*100}%')

Total data:  488389
Label: 0 484337 | 99.17033348416938%
Label: 1 4052 | 0.8296665158306186%


In [9]:
df.describe()

,x,y,z,rx,ry,rz,vx,vy,vz,Fx,Fy,Fz,Mx,My,Mz
count,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000,488389.000000
mean,-0.137849,-0.509390,0.072550,0.000130,-0.000315,-0.000847,-0.000010,-0.000039,-0.000013,-0.407760,-0.126849,5.004162,0.008293,-0.014836,-0.007263
std,0.003038,0.003120,0.000065,0.001211,0.001278,0.008951,0.001148,0.001239,0.000543,0.734809,0.865585,1.016898,0.090700,0.097045,0.167474
min,-0.145131,-0.517063,0.068158,-0.012374,-0.012927,-0.018829,-0.007014,-0.007297,-0.025372,-5.564121,-7.356778,-0.617198,-1.014954,-1.371579,-0.366593
25%,-0.140127,-0.511813,0.072536,-0.000808,-0.001371,-0.009156,-0.001095,-0.001101,-0.000146,-0.998160,-0.853043,4.619214,-0.068185,-0.095233,-0.137568
50%,-0.137731,-0.509246,0.072552,0.000130,-0.000245,-0.004691,-0.000016,-0.000177,-0.000004,-0.377296,-0.011427,4.975387,0.022135,-0.012592,-0.002113
75%,-0.135496,-0.506941,0.072568,0.001151,0.000735,0.008329,0.001058,0.001062,0.000138,0.171690,0.611727,5.360380,0.086866,0.066398,0.092925
max,-0.131221,-0.502472,0.072921,0.005757,0.004014,0.019179,0.004261,0.006702,0.004106,6.415391,6.491598,14.144978,0.818818,0.749540,0.364645


In [10]:
# corr = df.corr()
# plt.figure(figsize=(15,10))
# sns.heatmap(corr,annot=True)
# plt.show()

In [11]:
# corr_with_case = corr['case']
# # delete .abs() if we do not want abs values
# sorted_corr = corr_with_case.abs().sort_values(ascending=False)

# corr_df = pd.DataFrame(sorted_corr)

# # rename the column
# corr_df = corr_df.rename(columns={'case': 'correlation'})

In [12]:
# corr_df

### Data Preprocessing

In [13]:
feature_names = ['Fx', 'Fy', 'Fz', 'Mx', 'My']
# feature_names = ['fx', 'fy', 'fz', 'mx', 'my']

X_df = df[feature_names]
X = X_df.to_numpy()
y = df.Case.to_numpy()

window_len = 300

if window_len:
    n_features = len(feature_names)  # 3

    row = X.shape[0] + 1 - window_len
    col_len = n_features * window_len
    new_x = np.zeros((row, col_len))
    new_y = np.zeros((row, 1))

    for i in range(len(new_x)):
        new_x[i] = X[i:i + window_len].reshape(1, col_len)[0][::-1]
        new_y[i] = y[i + (window_len - 1)]

    y = deepcopy(new_y)
    X = deepcopy(new_x)

total = len(y)
pos = int(y.sum())
neg = total - pos
weight = neg/pos # 13.5

In [14]:
weight

119.45656465942744

In [15]:
X_df.tail()

,Fx,Fy,Fz,Mx,My
488384,-1.160801,-0.431394,4.523915,0.001261,0.100709
488385,-1.041252,-0.497221,4.418812,-0.002497,0.106457
488386,-0.745114,-0.498609,4.493353,-0.009804,0.116767
488387,-1.114615,-0.377344,4.280449,-0.014688,0.113422
488388,-1.069373,-0.256660,4.678466,-0.018297,0.113746


In [16]:
X.shape

(488090, 1500)

In [17]:
print(f"Total Dataset size: {X.shape}")
print(f"Ratio of 1's in dataset {(pos/total) * 100}% and count is: {pos}")
print(f"Ratio of 0's in dataset {(neg/total) * 100}% and count is: {neg}")

Total Dataset size: (488090, 1500)
Ratio of 1's in dataset 0.8301747628511136% and count is: 4052
Ratio of 0's in dataset 99.16982523714888% and count is: 484038


### Helper Functions

In [18]:
def plot_prc_and_roc(y_true, y_pred):

    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
    average_precision = average_precision_score(y_true, y_pred)
    
    plt.figure()
    plt.step(recalls, precisions, color='b', alpha=1.0, where='post')
    plt.fill_between(recalls, precisions, step='post', alpha=0.1, color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.grid()

    plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
              average_precision))
    plt.show()
    
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    areaUnderROC = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic: \
            Area under the curve = {0:0.2f}'.format(areaUnderROC))
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()       


In [19]:
def plot_precision_recall_vs_threshold(y_true, y_pred):
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
    f1_vec = 2 * (precisions * recalls) / (precisions + recalls)
    plt.figure()
    plt.plot(thresholds, precisions[:-1], "b--", label='Precision')
    plt.plot(thresholds, recalls[:-1], "g-", label='Recall')
    plt.plot(thresholds, f1_vec[:-1], "k-", label='F1')
    plt.xlabel('Threshold')
    plt.title('Precision/ Recall/ F1 Score/ Threshold')
    plt.legend()
    plt.grid()
    plt.show()

In [20]:
def xgb_stratified_kfold_cross_val(X, y, model, print_info):
    """
    StratifiedKFold cross-validation. 
    """
    skf = StratifiedKFold(n_splits=10, shuffle=False)
    aucpr_scores = []
    roc_scores = []
    prec_scores =  []
    recall_scores = []

    for fold_id, (train_index, valid_index) in enumerate(skf.split(X, y)):
        print(f"Fold {fold_id}:")
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if print_info:
            print(f"Ratio of 1's in train set {np.round((y_train.sum())/len(y_train),4) * 100}%")
            print(f"Ratio of 1's in validation set {np.round((y_valid.sum()/len(y_valid)),4) * 100}%")
        
           # fit model
        model.fit(
            X_train,
            y_train,
            early_stopping_rounds=15,
            eval_metric='aucpr', #auc
            eval_set=[(X_valid, y_valid)],
            verbose=False)
        
        y_val_pred = model.predict_proba(X_valid)[:, 1]
        roc = roc_auc_score(y_valid, y_val_pred)
        aucpr = average_precision_score(y_valid, y_val_pred)
        prec = precision_score(y_valid, y_val_pred > 0.5)
        recall = recall_score(y_valid, y_val_pred > 0.5)
        if print_info:
            print('--------------------------------------')
            print(f"Aucpr score for fold {fold_id} is : {aucpr}")
            print(f"ROC score for fold {fold_id} is : {roc}")
            print(f"Precision  : {prec}")
            print(f"Recall : {recall}")
            print('--------------------------------------')

        aucpr_scores.append(aucpr)
        roc_scores.append(roc)
        prec_scores.append(prec)
        recall_scores.append(recall)


    print()
    print(f"Mean across all folds:\nAUCPR: {np.mean(aucpr_scores)}\nROC {np.mean(roc_scores)}\nPrecision: {np.mean(prec_scores)}\nRecall: {np.mean(recall_scores)}")


In [21]:
def get_scores_thresholds(X_test, y_test, model, algo=None):
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for thresh in thresholds:
        if algo == "XGB":
            y_pred = model.predict_proba(X_test)[:, 1]
        else:
            y_pred = model.predict(X_test)
            
        print('threshold:', thresh)
        print(confusion_matrix(y_test, y_pred > thresh))
        print(classification_report(y_test, y_pred > thresh))
        print('--------------------------------------')

In [22]:
def xgb_plot(result, string='aucpr'):
    # plot learning curves
    plt.plot(results['validation_0'][string], label='train')
    plt.plot(results['validation_1'][string], label='test')
    # show the legend
    plt.legend()
    plt.grid()
    plt.show()

### XGboost baseline

### Based on GridSearchCv

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
print('\nAfter split')
print(f"Ratio of 1's in train set {(y_train.sum())/len(y_train) * 100}")
print(f"Ratio of 1's in test set {(y_test.sum()/len(y_test)) * 100}")


After split
Ratio of 1's in train set 0.8302772029748612
Ratio of 1's in test set 0.8297650023561228


In [24]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic',
                        n_estimators=800,
                        max_depth=5,
                        subsample=0.8, colsample_bytree=0.8,
                        seed=42, 
                        early_stopping_rounds=20,
                        eval_metric='aucpr',#['aucpr','logloss'], #auc
                        n_jobs = -1)


clf_xgb.fit(X_train, 
            y_train,
            # we evaluate on both train and test set
            eval_set=[(X_train, y_train),(X_test, y_test)],
            verbose=1)
# xgb_stratified_kfold_cross_val(X, y, clf_xgb, print_info=False)

[0]	validation_0-aucpr:0.14266	validation_1-aucpr:0.10861
[1]	validation_0-aucpr:0.18533	validation_1-aucpr:0.13175
[2]	validation_0-aucpr:0.21711	validation_1-aucpr:0.15120
[3]	validation_0-aucpr:0.23766	validation_1-aucpr:0.18086
[4]	validation_0-aucpr:0.26987	validation_1-aucpr:0.20257
[5]	validation_0-aucpr:0.28361	validation_1-aucpr:0.21642
[6]	validation_0-aucpr:0.31475	validation_1-aucpr:0.23610
[7]	validation_0-aucpr:0.33163	validation_1-aucpr:0.24798
[8]	validation_0-aucpr:0.35100	validation_1-aucpr:0.26006
[9]	validation_0-aucpr:0.37925	validation_1-aucpr:0.28601
[10]	validation_0-aucpr:0.39905	validation_1-aucpr:0.30762
[11]	validation_0-aucpr:0.41358	validation_1-aucpr:0.31608
[12]	validation_0-aucpr:0.44133	validation_1-aucpr:0.33420
[13]	validation_0-aucpr:0.46111	validation_1-aucpr:0.35163
[14]	validation_0-aucpr:0.47345	validation_1-aucpr:0.36230
[15]	validation_0-aucpr:0.48701	validation_1-aucpr:0.36970
[16]	validation_0-aucpr:0.50173	validation_1-aucpr:0.38786
[17]	va

[139]	validation_0-aucpr:0.99983	validation_1-aucpr:0.87839
[140]	validation_0-aucpr:0.99985	validation_1-aucpr:0.87887
[141]	validation_0-aucpr:0.99988	validation_1-aucpr:0.88047
[142]	validation_0-aucpr:0.99992	validation_1-aucpr:0.88373
[143]	validation_0-aucpr:0.99993	validation_1-aucpr:0.88438
[144]	validation_0-aucpr:0.99993	validation_1-aucpr:0.88440
[145]	validation_0-aucpr:0.99993	validation_1-aucpr:0.88476
[146]	validation_0-aucpr:0.99993	validation_1-aucpr:0.88503
[147]	validation_0-aucpr:0.99995	validation_1-aucpr:0.88522
[148]	validation_0-aucpr:0.99996	validation_1-aucpr:0.88815
[149]	validation_0-aucpr:0.99997	validation_1-aucpr:0.88917
[150]	validation_0-aucpr:0.99998	validation_1-aucpr:0.88994
[151]	validation_0-aucpr:0.99998	validation_1-aucpr:0.89050
[152]	validation_0-aucpr:0.99999	validation_1-aucpr:0.89065
[153]	validation_0-aucpr:0.99999	validation_1-aucpr:0.89130
[154]	validation_0-aucpr:0.99999	validation_1-aucpr:0.89196
[155]	validation_0-aucpr:0.99999	validat

[276]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93429
[277]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93477
[278]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93508
[279]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93525
[280]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93527
[281]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93557
[282]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93567
[283]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93551
[284]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93562
[285]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93558
[286]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93571
[287]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93577
[288]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93596
[289]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93615
[290]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93613
[291]	validation_0-aucpr:1.00000	validation_1-aucpr:0.93614
[292]	validation_0-aucpr:1.00000	validat

[413]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94653
[414]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94661
[415]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94644
[416]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94648
[417]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94667
[418]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94676
[419]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94691
[420]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94689
[421]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94693
[422]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94690
[423]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94724
[424]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94737
[425]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94747
[426]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94752
[427]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94754
[428]	validation_0-aucpr:1.00000	validation_1-aucpr:0.94743
[429]	validation_0-aucpr:1.00000	validat

[550]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95193
[551]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95201
[552]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95216
[553]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95231
[554]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95245
[555]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95248
[556]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95249
[557]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95246
[558]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95265
[559]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95271
[560]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95279
[561]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95287
[562]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95299
[563]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95289
[564]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95294
[565]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95307
[566]	validation_0-aucpr:1.00000	validat

[687]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95526
[688]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95524
[689]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95527
[690]	validation_0-aucpr:1.00000	validation_1-aucpr:0.95518


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=20,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=800, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [25]:
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]
print("Perfect confusion matrix")
print(confusion_matrix(y_test, y_test > 0.5))
print('Test set confusion matrix')
print(confusion_matrix(y_test, y_test_pred > 0.5))

Perfect confusion matrix
[[96808     0]
 [    0   810]]
Test set confusion matrix
[[96789    19]
 [  240   570]]


In [26]:
# clf_xgb.save_model("xgboost_feb1_partial.json")

In [28]:
X_df.columns

Index(['Fx', 'Fy', 'Fz', 'Mx', 'My'], dtype='object')

In [35]:
X.shape

(488090, 1500)

In [36]:
# importances = pd.DataFrame({
#     'Feature': X_df,
#     'Importance': clf_xgb.feature_importances_
# })
# importances = importances.sort_values(by='Importance', ascending=False)
# importances = importances.set_index('Feature')
# importances


In [27]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', 
                        seed=42, 
                        n_estimators=230,
                        eval_metric='aucpr',#['aucpr','logloss'], #auc
                        n_jobs = -1)
clf_xgb.fit(X, 
            y,
            verbose=1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=230, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [28]:
y_pred = clf_xgb.predict_proba(X)[:, 1]
print("Perfect confusion matrix")
print(confusion_matrix(y, y > 0.5))
print('Test set confusion matrix')
print(confusion_matrix(y, y_pred > 0.5))

Perfect confusion matrix
[[165919      0]
 [     0   3299]]
Test set confusion matrix
[[165919      0]
 [     0   3299]]


In [29]:
# clf_xgb.save_model("xgboost_sim_full.json")

### Optimizing XGBoost

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train_len = len(X_train)
X_test_len = len(X_test)
y_train_len = len(y_train)
y_test_len = len(y_test)

subset = 10 # take 10% of all data

idx_x_train = int(X_train_len/subset)
idx_x_test = int(X_test_len/subset)
idx_y_train = int(y_train_len/subset)
idx_y_test = int(y_test_len/subset)

X_train_subset =  X_train[:idx_x_train,:]
X_test_subset = X_test[:idx_x_test,:]
y_train_subset = y_train[:idx_y_train,:]
y_test_subset = y_test[:idx_y_test,:]

print(f"X_train_subset {X_train_subset.shape} and original was {X_train.shape}")
print(f"X_test_subset {X_test_subset.shape} and original was {X_test.shape}")
print(f"y_train_subset {y_train_subset.shape} and original was {y_train.shape}")
print(f"y_test_subset {y_test_subset.shape} and original was {y_test.shape}")

In [ ]:
weight

In [ ]:
param_grid = {
    'max_depth': [3, 4, 5, 8],
    'learning_rate': [0.1, 0.001, 0.05],
    'gamma': [0, 0.25, 1.0],
    'reg_lambda': [0, 1.0, 10.0]}

In [ ]:
# scoring
# https://scikit-learn.org/stable/modules/model_evaluation.html 
xgb_estimator=xgb.XGBClassifier(objective ='binary:logistic', 
                                seed=42, 
                                subsample=0.9,
                                colsample_bytree=0.5,
                                early_stopping_rounds=20,
                                eval_metric='aucpr')

                        
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

optimal_params = RandomizedSearchCV(estimator=xgb_estimator, 
                           param_distributions=param_grid, 
                           n_iter=2, # test randomly 48 combinations
                           scoring='roc_auc', 
                           refit='roc_auc', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=2)

In [ ]:
optim_results = optimal_params.fit(X_train_subset,
                               y_train_subset,
                               eval_set =[(X_test_subset, y_test_subset)],
                               verbose=True)

In [ ]:
optim_results.best_estimator_


In [ ]:
optim_results.best_score_

In [ ]:
optim_results.best_params_

In [ ]:
weight

### After Search of params

In [ ]:


clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', 
                        scale_pos_weight = 1,
                        max_depth = 8,
                        learning_rate=0.05,
                        seed=42, 
                        early_stopping_rounds=20,
                        eval_metric='aucpr',#['aucpr','logloss'], #auc
                        n_jobs=-1)

clf_xgb.fit(X_train, 
            y_train,
            # we evaluate on both train and test set
            eval_set=[(X_train, y_train),(X_test, y_test)],
            verbose=1)

In [ ]:
results = clf_xgb.evals_result()

In [ ]:
xgb_plot(result=results,string='aucpr')

In [ ]:
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]
print("Perfect confusion matrix")
print(confusion_matrix(y_test, y_test > 0.5))
print('Test set confusion matrix')
print(confusion_matrix(y_test, y_test_pred > 0.5))

### Trying with class weights

In [ ]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', 
                        scale_pos_weight = weight,
                        max_depth = 8,
                        learning_rate=0.05,
                        seed=42, 
                        early_stopping_rounds=20,
                        eval_metric='aucpr',#['aucpr','logloss'], #auc
                        n_jobs=-1)

clf_xgb.fit(X_train, 
            y_train,
            # we evaluate on both train and test set
            eval_set=[(X_train, y_train),(X_test, y_test)],
            verbose=1)

In [ ]:
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]
print("Perfect confusion matrix")
print(confusion_matrix(y_test, y_test > 0.5))
print('Test set confusion matrix')
print(confusion_matrix(y_test, y_test_pred > 0.5))